In [25]:
import numpy as np
import pandas as pd
import arcpy
import random
from arcpy.sa import *
from arcpy import env

In [26]:
arcpy.CheckOutExtension("Spatial")
# Check and enable the Geostatistical Analyst extension
if arcpy.CheckExtension("GeoStats") == "Available":
    arcpy.CheckOutExtension("GeoStats")
else:
    raise Exception("Geostatistical Analyst Extension is not available.")

In [ ]:
print(arcpy.env.workspace)

# Split the raw data into training set (train) and test set (test).
1. This study primarily uses a ~10% ratio to split the data into train and test sets, with the test dataset being obtained through spatial stratified random sampling method, based on internal basins as the characteristic.
2. Considering the total number of samples (367) and the number of samples in each internal basin, we decided to obtain 38 samples as the test set, including Pattani (4), Malay (14), Penyu (1), West Natuna (3), Wan'an (4), Mekong (4), and Sarawak (8).

In [ ]:
# Split the dataset
all_data_points = "whole_data_points"
train_data = "train_data_points"
test_data = "test_data_points"

# Check if the output feature classes already exist
if arcpy.Exists(train_data):
    arcpy.Delete_management(train_data)
if arcpy.Exists(test_data):
    arcpy.Delete_management(test_data)
# Get the spatial reference of whole_data_points
spatial_ref = arcpy.Describe(all_data_points).spatialReference
    
# Create feature classes for the test and training sets
arcpy.CreateFeatureclass_management(arcpy.env.workspace, train_data, template=all_data_points, spatial_reference=spatial_ref)
arcpy.CreateFeatureclass_management(arcpy.env.workspace, test_data, template=all_data_points, spatial_reference=spatial_ref)

# Perform stratified random sampling by basin and set a fixed ratio (~10% for test)
sample_counts = {
    "Pattani": 4,
    "Malay": 14,
    "Penyu": 1,
    "West Natuna": 3,
    "Wan'an": 4,
    "Mekong": 4,
    "Sarawak": 8
}
# Save the OID list of test samples, used to differentiate the training set
test_oids = []
# Perform sampling by internal basin
for basin, count in sample_counts.items():
    

    safe_basin = basin.replace("'", "''")  
    

    query = f"\"Basin\" = '{safe_basin}'"
    

    arcpy.MakeFeatureLayer_management(all_data_points, "Basin_layer", query)
    

    oids = [row[0] for row in arcpy.da.SearchCursor("Basin_layer", "OID@")]
    

    sample_oids = random.sample(oids, min(count, len(oids)))  
    test_oids.extend(sample_oids)
    
    # Copy the sampled points to the test feature class
    for oid in sample_oids:
        arcpy.SelectLayerByAttribute_management("Basin_layer", "NEW_SELECTION", f"OBJECTID = {oid}")
        arcpy.Append_management("Basin_layer", test_data, "NO_TEST")
    

    arcpy.SelectLayerByAttribute_management("Basin_layer", "CLEAR_SELECTION")
    
    # Delete the temporary layer
    arcpy.Delete_management("Basin_layer")
    

# Create the training set
arcpy.MakeFeatureLayer_management(all_data_points, "train_layer")
test_oid_string = ', '.join(map(str, test_oids))  
arcpy.SelectLayerByAttribute_management("train_layer", "NEW_SELECTION", f"OBJECTID NOT IN ({test_oid_string})")
arcpy.Append_management("train_layer", train_data, "NO_TEST")
# Delete the temporary training set layer
arcpy.Delete_management("train_layer")

# For the OK (Ordinary Kriging) Standard 10-fold CV (Cross-Validation)

In [5]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import KFold

# Modify the original data to obtain the corresponding two columns of data after quantile transformation.
input_feature_class = "train_data_points"
target_field_MCO = "MCO_SD_TOC_Mt_km2_"
target_field_MMCT = "MMCT_SD_TOC_Mt_km2_"

# Define the list of target fields that need to be transformed
target_fields = [target_field_MCO, target_field_MMCT]

# Create dictionaries to store the transformed point data and quantile transformers
transformed_points = {}
quantile_transformers = {}

# Perform quantile transformation for each target field
for target_field in target_fields:
    print(f"Processing field: {target_field}")

    # Extract data
    points = [(row[0], row[1], row[2].centroid) for row in arcpy.da.SearchCursor(input_feature_class, ["OBJECTID", target_field, "SHAPE@"])]
    actual_values = np.array([point[1] for point in points]).reshape(-1, 1)

    # Quantile transformation
    qt = QuantileTransformer(output_distribution="normal", random_state=42)
    transformed_values = qt.fit_transform(actual_values).flatten()

    # Add quantile transformed field
    qt_field_name = f"QT_{target_field}"
    if qt_field_name not in [field.name for field in arcpy.ListFields(input_feature_class)]:
        arcpy.AddField_management(input_feature_class, qt_field_name, "DOUBLE")

    # Write the transformed values
    with arcpy.da.UpdateCursor(input_feature_class, ["OBJECTID", qt_field_name]) as cursor:
        for row in cursor:
            obj_id = row[0]
            transformed_value = next((val for oid, val in zip([p[0] for p in points], transformed_values) if oid == obj_id), None)
            if transformed_value is not None:
                row[1] = transformed_value
                cursor.updateRow(row)

    # Store the quantile transformer and transformed point data
    quantile_transformers[target_field] = qt
    transformed_points[target_field] = [(p[0], p[1], p[2], val) for p, val in zip(points, transformed_values)]

print("Quantile Transformation completed.")


def prepare_training_and_validation_layers(transformed_points, target_field, input_feature_class, train_idx, val_idx):
    """
    Generate training and validation layers based on the specified training and validation indices.
    """
    points = transformed_points[target_field]
    train_fc = f"Train_{target_field}"
    val_fc = f"Validation_{target_field}"

    # Extract training and validation points
    train_points = [points[i] for i in train_idx]
    val_points = [points[i] for i in val_idx]

    # Generate the training layer
    where_clause_train = f"OBJECTID IN ({','.join(str(p[0]) for p in train_points)})"
    arcpy.Select_analysis(input_feature_class, train_fc, where_clause_train)

    # Generate the validation layer
    where_clause_val = f"OBJECTID IN ({','.join(str(p[0]) for p in val_points)})"
    arcpy.Select_analysis(input_feature_class, val_fc, where_clause_val)

    return train_fc, val_fc


def standard_kfold_cv_kriging(transformed_points, quantile_transformers, target_field, kriging_model_name, input_feature_class, n_splits=10):
    """
    OK (Ordinary Kriging) interpolation evaluation function based on 10-fold Cross-Validation (CV).
    """
    # Store error results
    mae_list = []
    rmse_list = []
    sse = 0

    points = transformed_points[target_field]
    qt = quantile_transformers[target_field]
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    point_count = len(points)

    for fold, (train_idx, val_idx) in enumerate(kf.split(points)):
        print(f"Processing fold {fold + 1}/{n_splits}...")

        # Generate training and validation layers
        train_fc, val_fc = prepare_training_and_validation_layers(transformed_points, target_field, input_feature_class, train_idx, val_idx)

        # Perform Kriging interpolation on the training set
        kriging_model = KrigingModelOrdinary(kriging_model_name)
        kriging_result = Kriging(train_fc, f"QT_{target_field}", kriging_model)

        # Iterate through the validation set for predictions
        for idx in val_idx:
            test_point = points[idx]
            original_location = test_point[2]

            predicted_value_raw = arcpy.GetCellValue_management(kriging_result, f"{original_location.X} {original_location.Y}", "1").getOutput(0)

            # Check if it's NoData
            if predicted_value_raw == "NoData":
                print(f"Warning: NoData encountered for point {idx + 1}/{point_count}. Skipping this point.")
                continue

            predicted_value_transformed = float(predicted_value_raw)

            # Inverse quantile transformation
            predicted_value = qt.inverse_transform([[predicted_value_transformed]])[0][0]
            actual_value = test_point[1]

            # Calculate error and squared error
            error = predicted_value - actual_value
            squared_error = error ** 2

            mae_list.append(abs(error))
            rmse_list.append(squared_error)
            sse += squared_error

        # Delete temporary layers
        arcpy.Delete_management(train_fc)
        arcpy.Delete_management(val_fc)

    # Calculate MAE, RMSE, and R2
    mae = np.mean(mae_list)
    rmse = np.sqrt(np.mean(rmse_list))
    mean_actual = np.mean([p[1] for p in points])
    sst = sum((val - mean_actual) ** 2 for val in [p[1] for p in points])
    r2 = 1 - (sse / sst)

    return mae, rmse, r2




Processing field: MCO_SD_TOC_Mt_km2_


G:\Arcgis_pro\bin\Python\envs\arcgispro-py3-clone\Lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (329). n_quantiles is set to n_samples.
  warnings.warn(


Processing field: MMCT_SD_TOC_Mt_km2_


G:\Arcgis_pro\bin\Python\envs\arcgispro-py3-clone\Lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (329). n_quantiles is set to n_samples.
  warnings.warn(


Quantile Transformation completed.
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
K-fold CV Results - MAE: 3.8650, RMSE: 6.1174, R2: 0.2887


In [6]:
# The target value is MCO, standard LOOCV

input_feature_class = "train_data_points"  
target_field = "MCO_SD_TOC_Mt_km2_"  
print(target_field)

# Define the list of Kriging models
kriging_models = ["SPHERICAL", "EXPONENTIAL", "GAUSSIAN", "LINEAR", "CIRCULAR"]

mae_results = {}
rmse_results = {}
r2_results = {}


for kriging_model_name in kriging_models:
    print(f"Processing Kriging model: {kriging_model_name}")
    

    mae, rmse, r2 = standard_kfold_cv_kriging(
        transformed_points,
        quantile_transformers,
        target_field,
        kriging_model_name,
        input_feature_class,
        n_splits=10  
    )
    

    mae_results[kriging_model_name] = mae
    rmse_results[kriging_model_name] = rmse
    r2_results[kriging_model_name] = r2


for model_name in kriging_models:
    print(f"{model_name} MAE: {mae_results[model_name]}")
    print(f"{model_name} RMSE: {rmse_results[model_name]}")
    print(f"{model_name} R2: {r2_results[model_name]}")

arcpy.CheckInExtension("Spatial")

MCO_SD_TOC_Mt_km2_
Processing Kriging model: SPHERICAL
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing Kriging model: EXPONENTIAL
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing Kriging model: GAUSSIAN
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing Kriging model: LINEAR
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10

'CheckedOut'

In [7]:
# The target value is MMCT, standard LOOCV

input_feature_class = "train_data_points"  
target_field = "MMCT_SD_TOC_Mt_km2_"  
print(target_field)

# Define the list of Kriging models
kriging_models = ["SPHERICAL", "EXPONENTIAL", "GAUSSIAN", "LINEAR", "CIRCULAR"]

mae_results = {}
rmse_results = {}
r2_results = {}


for kriging_model_name in kriging_models:
    print(f"Processing Kriging model: {kriging_model_name}")
    

    mae, rmse, r2 = standard_kfold_cv_kriging(
        transformed_points,
        quantile_transformers,
        target_field,
        kriging_model_name,
        input_feature_class,
        n_splits=10  
    )
    

    mae_results[kriging_model_name] = mae
    rmse_results[kriging_model_name] = rmse
    r2_results[kriging_model_name] = r2


for model_name in kriging_models:
    print(f"{model_name} MAE: {mae_results[model_name]}")
    print(f"{model_name} RMSE: {rmse_results[model_name]}")
    print(f"{model_name} R2: {r2_results[model_name]}")


arcpy.CheckInExtension("Spatial")

MMCT_SD_TOC_Mt_km2_
Processing Kriging model: SPHERICAL
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing Kriging model: EXPONENTIAL
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing Kriging model: GAUSSIAN
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing Kriging model: LINEAR
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/1

'CheckedOut'

# Standard 10-fold CV for IDW

In [8]:
from arcpy.sa import Idw
from sklearn.model_selection import KFold

def prepare_data(input_fc, target_field):

    points = [(row[0], row[1].centroid, row[2]) for row in arcpy.da.SearchCursor(input_fc, ["OBJECTID", "SHAPE@", target_field])]
    return points

def prepare_training_and_validation_layers(points, target_field, input_feature_class, train_idx, val_idx):
    """
    Generate training and validation layers based on the specified training and validation indices.
    """
    train_fc = f"Train_{target_field}"
    val_fc = f"Validation_{target_field}"

    train_points = [points[i] for i in train_idx]
    val_points = [points[i] for i in val_idx]


    where_clause_train = f"OBJECTID IN ({','.join(str(p[0]) for p in train_points)})"
    arcpy.Select_analysis(input_feature_class, train_fc, where_clause_train)


    where_clause_val = f"OBJECTID IN ({','.join(str(p[0]) for p in val_points)})"
    arcpy.Select_analysis(input_feature_class, val_fc, where_clause_val)

    return train_fc, val_fc

def kfold_cv_idw(points, target_field, decay_power, input_feature_class, n_splits=10):
    """
    10-Fold CV based IDW interpolation evaluation function.
    """
    print(f"Train the IDW of decay power: {decay_power}...")
    

    mae_list = []
    rmse_list = []
    sse = 0

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    point_count = len(points)

    for fold, (train_idx, val_idx) in enumerate(kf.split(points)):
        print(f"Processing fold {fold + 1}/{n_splits}...")


        train_fc, val_fc = prepare_training_and_validation_layers(points, target_field, input_feature_class, train_idx, val_idx)


        idw_result = Idw(train_fc, target_field, power=decay_power)


        for idx in val_idx:
            test_point = points[idx]
            test_point_location = test_point[1]
            
            predicted_value_raw = arcpy.GetCellValue_management(
                idw_result, f"{test_point_location.X} {test_point_location.Y}", "1"
            ).getOutput(0)


            if predicted_value_raw == "NoData":
                print(f"Warning: NoData encountered for point {idx + 1}/{point_count}. Skipping this point.")
                continue

            predicted_value = float(predicted_value_raw)
            actual_value = test_point[2]


            error = predicted_value - actual_value
            mae_list.append(abs(error))
            rmse_list.append(error ** 2)
            sse += error ** 2


        arcpy.Delete_management(train_fc)
        arcpy.Delete_management(val_fc)


    mae = np.mean(mae_list)
    rmse = np.sqrt(np.mean(rmse_list))
    mean_actual = np.mean([p[2] for p in points])
    sst = sum((val - mean_actual) ** 2 for val in [p[2] for p in points])
    r2 = 1 - (sse / sst)

    return mae, rmse, r2

In [9]:
# Perform K-Fold CV for IDW, with the target value being MCO.
input_feature_class = "train_data_points"  
target_field = "MCO_SD_TOC_Mt_km2_"  
decay_powers = [1, 2, 3, 4]  # Replace with the list of decay exponents to be evaluated，here is 1,2,3,4

points = prepare_data(input_feature_class, target_field)


mae_results = {}
rmse_results = {}
r2_results = {}

for decay_power in decay_powers:
    mae, rmse, r2 = kfold_cv_idw(points, target_field, decay_power, input_feature_class, n_splits=10)
    mae_results[decay_power] = mae
    rmse_results[decay_power] = rmse
    r2_results[decay_power] = r2


for power in decay_powers:
    print(f"Decay Power {power} - MAE: {mae_results[power]}, RMSE: {rmse_results[power]}, R2: {r2_results[power]}")


arcpy.CheckInExtension("Spatial")

Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Decay Power 1 - MAE: 3.0095647335011

'CheckedOut'

In [10]:
# Perform K-Fold CV for IDW, with the target value being MMCT.
input_feature_class = "train_data_points"  
target_field = "MMCT_SD_TOC_Mt_km2_"  
decay_powers = [1, 2, 3, 4]  

points = prepare_data(input_feature_class, target_field)

# 评估每种衰减幂次
mae_results = {}
rmse_results = {}
r2_results = {}

for decay_power in decay_powers:
    mae, rmse, r2 = kfold_cv_idw(points, target_field, decay_power, input_feature_class, n_splits=10)
    mae_results[decay_power] = mae
    rmse_results[decay_power] = rmse
    r2_results[decay_power] = r2


for power in decay_powers:
    print(f"Decay Power {power} - MAE: {mae_results[power]}, RMSE: {rmse_results[power]}, R2: {r2_results[power]}")

arcpy.CheckInExtension("Spatial")

Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Decay Power 1 - MAE: 1.2042436101929

'CheckedOut'

# Standard 10-fold CV for Thiessen.
This is actually not necessary, as there are no hyperparameters to tune. The purpose here is to compare with other methods (IDW, OK, and RF) based on 10-fold CV.

In [11]:
from sklearn.model_selection import KFold
from arcpy.sa import *

def prepare_training_and_validation_layers_thiessen(points, target_field, input_feature_class, train_idx, val_idx):
    """
    Generate training and validation layers based on the specified training and validation indices.
    """
    train_fc = f"Train_{target_field}"
    val_fc = f"Validation_{target_field}"


    train_points = [points[i] for i in train_idx]
    val_points = [points[i] for i in val_idx]


    where_clause_train = f"OBJECTID IN ({','.join(str(p[0]) for p in train_points)})"
    arcpy.Select_analysis(input_feature_class, train_fc, where_clause_train)


    where_clause_val = f"OBJECTID IN ({','.join(str(p[0]) for p in val_points)})"
    arcpy.Select_analysis(input_feature_class, val_fc, where_clause_val)

    return train_fc, val_fc

def standard_kfold_cv_thiessen(points, target_field, input_feature_class, n_splits=10):
    """
    10-Fold CV based Thiessen interpolation evaluation function.
    """
    mae_list = []
    rmse_list = []
    sse = 0

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    point_count = len(points)

    for fold, (train_idx, val_idx) in enumerate(kf.split(points)):
        print(f"Processing fold {fold + 1}/{n_splits}...")


        train_fc, val_fc = prepare_training_and_validation_layers_thiessen(points, target_field, input_feature_class, train_idx, val_idx)


        thiessen_output = f"Thiessen_{fold + 1}"
        arcpy.CreateThiessenPolygons_analysis(train_fc, thiessen_output, "ALL")


        joined_output = f"Joined_{fold + 1}"
        arcpy.SpatialJoin_analysis(thiessen_output, train_fc, joined_output, "JOIN_ONE_TO_ONE", "KEEP_ALL", match_option="CONTAINS")


        for idx in val_idx:
            test_point = points[idx]
            original_location = test_point[1]

            with arcpy.da.SearchCursor(joined_output, [target_field, "SHAPE@"]) as cursor:
                predicted_value = None
                for row in cursor:
                    if row[1].distanceTo(original_location) < 1e-6:
                        predicted_value = row[0]
                        break

                if predicted_value is None:
                    print(f"Warning: Point {idx + 1}/{point_count} outside Thiessen polygons. Skipping.")
                    continue

            actual_value = test_point[2]


            error = predicted_value - actual_value
            squared_error = error ** 2

            mae_list.append(abs(error))
            rmse_list.append(squared_error)
            sse += squared_error


        arcpy.Delete_management(train_fc)
        arcpy.Delete_management(val_fc)
        arcpy.Delete_management(thiessen_output)
        arcpy.Delete_management(joined_output)


    mae = np.mean(mae_list)
    rmse = np.sqrt(np.mean(rmse_list))
    mean_actual = np.mean([p[2] for p in points])
    sst = sum((val - mean_actual) ** 2 for val in [p[2] for p in points])
    r2 = 1 - (sse / sst)

    return mae, rmse, r2

In [12]:
# Standard 10-fold CV for Thiessen interpolation, with the target being MCO.
input_feature_class = "train_data_points"  
target_field = "MCO_SD_TOC_Mt_km2_"  


points = [(row[0], row[1].centroid, row[2]) for row in arcpy.da.SearchCursor(input_feature_class, ["OBJECTID", "SHAPE@", target_field])]


mae, rmse, r2 = standard_kfold_cv_thiessen(points, target_field, input_feature_class, n_splits=10)


print(f"Thiessen 10-Fold CV Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


arcpy.CheckInExtension("Spatial")

Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Thiessen 10-Fold CV Results - MAE: 3.1490, RMSE: 5.4312, R2: 0.4341


'CheckedOut'

In [13]:
# Standard 10-fold CV for Thiessen interpolation, with the target being MMCT.
input_feature_class = "train_data_points"  
target_field = "MMCT_SD_TOC_Mt_km2_"  


points = [(row[0], row[1].centroid, row[2]) for row in arcpy.da.SearchCursor(input_feature_class, ["OBJECTID", "SHAPE@", target_field])]


mae, rmse, r2 = standard_kfold_cv_thiessen(points, target_field, input_feature_class, n_splits=10)


print(f"Thiessen 10-Fold CV Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


arcpy.CheckInExtension("Spatial")

Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Thiessen 10-Fold CV Results - MAE: 1.3412, RMSE: 2.4360, R2: 0.1162


'CheckedOut'

# Standard 10-fold CV for Random Forest.

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KDTree
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import arcpy

# Create a KDTree to facilitate the calculation of the values and distances of neighboring points.
whole_data_class = "whole_data_points"
whole_fields = ["Longitude___", "Latitude___", "MCO_SD_TOC_Mt_km2_", "MMCT_SD_TOC_Mt_km2_"]
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)

def kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k, n_splits=10):
    """
    K-Fold CV based Random Forest evaluation function, dynamically updating the features of neighboring points.
    :param input_feature_class: Input feature class
    :param target_field: Target field
    :param params: Hyperparameters for Random Forest
    :param whole_df: DataFrame containing the complete data
    :param whole_tree: KDTree object for calculating neighboring points
    :param k: Number of nearest neighbors, here is 25
    :param n_splits: Number of splits, default is 10
    :return: MAE, RMSE, R2
    """

    fields = ["Longitude___", "Latitude___", "Distance_shoreline_m", target_field]
    data = [row for row in arcpy.da.SearchCursor(input_feature_class, fields)]
    df = pd.DataFrame(data, columns=fields)

    X = df.drop(columns=[target_field])
    y = df[target_field]


    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)


    predictions = []
    actual_values = []


    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Processing fold {fold + 1}/{n_splits}...")
        

        X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
        y_train, y_val = y.iloc[train_idx].copy(), y.iloc[val_idx].copy()


        train_coords = X_train[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_train)):
            distances, indices = whole_tree.query(train_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  # Exclude the point itself
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_train.loc[X_train.index[idx], f"distance_{j}"] = dist
                X_train.loc[X_train.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]

        val_coords = X_val[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_val)):
            distances, indices = whole_tree.query(val_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  # Exclude the point itself
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_val.loc[X_val.index[idx], f"distance_{j}"] = dist
                X_val.loc[X_val.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]

        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_train, y_train)


        y_pred = model.predict(X_val)
        predictions.extend(y_pred)
        actual_values.extend(y_val)


    mae = mean_absolute_error(actual_values, predictions)
    rmse = np.sqrt(mean_squared_error(actual_values, predictions))
    r2 = r2_score(actual_values, predictions)

    return mae, rmse, r2


In [ ]:
# Standard k-fold CV for Random Forest, target value is MCO
input_feature_class = "train_data_point_SpatialJoin"
target_field = "MCO_SD_TOC_Mt_km2_"


results = []

# Define parameter grid
param_grid = {
    "n_estimators": list(range(50, 501, 50)),  
    "max_features": list(range(3, 34, 3))  
}


for params in ParameterGrid(param_grid):
    print(f"Training with n_estimators: {params['n_estimators']}, max_features: {params['max_features']}")

    mae, rmse, r2 = kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k=25, n_splits=10)

    results.append({
        'n_estimators': params['n_estimators'],
        'max_features': params['max_features'],
        'avg_mae': mae,
        'avg_rmse': rmse,
        'avg_r2': r2
    })

    print(f"Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


df_results = pd.DataFrame(results)
output_file = "1211_RandomForest_10FoldCV_Results_MCO_25临近点.csv"
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

In [ ]:
# Standard k-fold CV for Random Forest, target value is MMCT
input_feature_class = "train_data_point_SpatialJoin"
target_field = "MMCT_SD_TOC_Mt_km2_"


results = []


param_grid = {
    "n_estimators": list(range(50, 501, 50)),  
    "max_features": list(range(3, 34, 3))  
}


for params in ParameterGrid(param_grid):
    print(f"Training with n_estimators: {params['n_estimators']}, max_features: {params['max_features']}")

    mae, rmse, r2 = kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k=25, n_splits=10)

    results.append({
        'n_estimators': params['n_estimators'],
        'max_features': params['max_features'],
        'avg_mae': mae,
        'avg_rmse': rmse,
        'avg_r2': r2
    })

    print(f"Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


df_results = pd.DataFrame(results)
output_file = "1211_RandomForest_10FoldCV_Results_MMCT_25临近点.csv"
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# Evaluate the performance of the optimal parameter combination of each model on the unseen test data.

In [3]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
import arcpy

In [8]:

input_feature_class_train = "train_data_point_SpatialJoin"
input_feature_class_test = "test_data_point_SpatialJoin"
MCO_target_field = "MCO_SD_TOC_Mt_km2_" 
MMCT_target_field = "MMCT_SD_TOC_Mt_km2_"

In [9]:
# Thiessen polygon evaluation function
def test_evaluate_thiessen(input_feature_class_train, input_feature_class_test, target_field):
    try:
        # Create Thiessen polygons
        thiessen_output = "thiessen_train"
        arcpy.CreateThiessenPolygons_analysis(input_feature_class_train, thiessen_output, "ALL")

        # Add actual values of the training data to the Thiessen polygons
        joined_output = "joined_thiessen"
        arcpy.SpatialJoin_analysis(thiessen_output, input_feature_class_train, joined_output, "JOIN_ONE_TO_ONE", "KEEP_ALL", match_option="CONTAINS")


        predictions = []
        actual_values = []

        # Iterate through the test data
        with arcpy.da.SearchCursor(input_feature_class_test, ["OBJECTID", "SHAPE@", target_field]) as test_cursor:
            for test_point in test_cursor:
                test_location = test_point[1].centroid
                actual_value = test_point[2]
                actual_values.append(actual_value)

                # Iterate through the Thiessen polygons and find the one containing the test point
                predicted_value = None
                with arcpy.da.SearchCursor(joined_output, [target_field, "SHAPE@"]) as thiessen_cursor:
                    for row in thiessen_cursor:
                        if row[1].contains(test_location):
                            predicted_value = row[0]
                            break

                if predicted_value is not None:
                    predictions.append(predicted_value)
                else:
                    raise ValueError(f"No polygon in joined_output contains the test point OBJECTID {test_point[0]}")


        predictions = np.array(predictions)
        actual_values = np.array(actual_values)

        mae = np.mean(np.abs(predictions - actual_values))
        rmse = np.sqrt(np.mean((predictions - actual_values) ** 2))
        sst = np.sum((actual_values - np.mean(actual_values)) ** 2)
        sse = np.sum((predictions - actual_values) ** 2)
        r2 = 1 - (sse / sst)

        print(f"Thiessen Polygon Evaluation on Test Data:{target_field}")
        print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


        arcpy.Delete_management(thiessen_output)
        arcpy.Delete_management(joined_output)

        return mae, rmse, r2

    except Exception as e:
        print(f"Error during Thiessen polygon evaluation: {e}")
        return None, None, None

In [10]:
# Evaluate the final results of Thiessen polygons for MCO
mae, rmse, r2 = test_evaluate_thiessen(input_feature_class_train, input_feature_class_test, MCO_target_field)

# Evaluate the final results of Thiessen polygons for MMCT
mae, rmse, r2 = test_evaluate_thiessen(input_feature_class_train, input_feature_class_test, MMCT_target_field)


Thiessen Polygon Evaluation on Test Data:MCO_SD_TOC_Mt_km2_
MAE: 3.4084, RMSE: 5.5886, R2: 0.4630
Thiessen Polygon Evaluation on Test Data:MMCT_SD_TOC_Mt_km2_
MAE: 1.4009, RMSE: 2.4968, R2: -0.1524


In [14]:
# Evaluation function for the best OK (Ordinary Kriging) model
from sklearn.preprocessing import QuantileTransformer

input_feature_class = "train_data_points"
target_field_MCO = "MCO_SD_TOC_Mt_km2_"
target_field_MMCT = "MMCT_SD_TOC_Mt_km2_"


target_fields = [target_field_MCO, target_field_MMCT]

# Create dictionaries to store transformed point data and quantile transformers
transformed_points = {}
quantile_transformers = {}


for target_field in target_fields:
    print(f"Processing field: {target_field}")


    points = [(row[0], row[1], row[2].centroid) for row in arcpy.da.SearchCursor(input_feature_class, ["OBJECTID", target_field, "SHAPE@"])]
    actual_values = np.array([point[1] for point in points]).reshape(-1, 1)


    qt = QuantileTransformer(output_distribution="normal", random_state=42)
    transformed_values = qt.fit_transform(actual_values).flatten()


    qt_field_name = f"QT_{target_field}"
    if qt_field_name not in [field.name for field in arcpy.ListFields(input_feature_class)]:
        arcpy.AddField_management(input_feature_class, qt_field_name, "DOUBLE")


    with arcpy.da.UpdateCursor(input_feature_class, ["OBJECTID", qt_field_name]) as cursor:
        for row in cursor:
            obj_id = row[0]
            transformed_value = next((val for oid, val in zip([p[0] for p in points], transformed_values) if oid == obj_id), None)
            if transformed_value is not None:
                row[1] = transformed_value
                cursor.updateRow(row)

    # Store the quantile transformers and transformed point data
    quantile_transformers[target_field] = qt
    transformed_points[target_field] = [(p[0], p[1], p[2], val) for p, val in zip(points, transformed_values)]

print("Quantile Transformation completed.")

def test_evaluate_kriging(input_feature_class_train, input_feature_class_test, target_field_QT,target_field, kriging_model_name):
    try:

        kriging_model = KrigingModelOrdinary(kriging_model_name)
        kriging_result = Kriging(input_feature_class_train, target_field_QT, kriging_model)
        print(f"{kriging_model_name} model created based on training data.")
        qt = quantile_transformers[target_field]

        predictions = []
        actual_values = []

        with arcpy.da.SearchCursor(input_feature_class_test, ["OBJECTID", "SHAPE@", target_field]) as test_cursor:
            for test_point in test_cursor:
                test_location = test_point[1].centroid
                
                actual_value = test_point[2]
                actual_values.append(actual_value)

                predicted_value_transformed = float(
                    arcpy.GetCellValue_management(
                        kriging_result, f"{test_location.X} {test_location.Y}", "1"
                    ).getOutput(0)
                )
                predicted_value = qt.inverse_transform([[predicted_value_transformed]])[0][0]
                predictions.append(predicted_value)

        predictions = np.array(predictions)
        actual_values = np.array(actual_values)

        mae = np.mean(np.abs(predictions - actual_values))
        rmse = np.sqrt(np.mean((predictions - actual_values) ** 2))
        sst = np.sum((actual_values - np.mean(actual_values)) ** 2)
        sse = np.sum((predictions - actual_values) ** 2)
        r2 = 1 - (sse / sst)

        print(f"{kriging_model_name} Evaluation on Test Data:{target_field}")
        print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

        return mae, rmse, r2

    except Exception as e:
        print(f"Error during {kriging_model_name} evaluation: {e}")
        return None, None, None

Processing field: MCO_SD_TOC_Mt_km2_


G:\Arcgis_pro\bin\Python\envs\arcgispro-py3-clone\Lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (329). n_quantiles is set to n_samples.
  warnings.warn(


Processing field: MMCT_SD_TOC_Mt_km2_


G:\Arcgis_pro\bin\Python\envs\arcgispro-py3-clone\Lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (329). n_quantiles is set to n_samples.
  warnings.warn(


Quantile Transformation completed.


In [15]:
# Evaluate the final results of MCO's OK, based on the best OK model for MCO obtained from 10-fold CV on the training set: EXPONENTIAL.
# Note that a special point here is that the input target_field is the data transformed by QT (Quantile Transformation).
input_feature_class_train_OK = "train_data_points"
input_feature_class_test_OK = "test_data_points"
MCO_best_krigng_model = "EXPONENTIAL"
MCO_target_field_QT = "QT_MCO_SD_TOC_Mt_km2_"
MCO_target_field = "MCO_SD_TOC_Mt_km2_" 

mae, rmse, r2 = test_evaluate_kriging(input_feature_class_train_OK, input_feature_class_test, MCO_target_field_QT,MCO_target_field, MCO_best_krigng_model)

# Evaluate the final results of MMCT's OK, based on the best OK model for MMCT obtained from 10-fold CV on the training set: SPHERICAL.
# Note that a special point here is that the input target_field is the data transformed by QT (Quantile Transformation).
MMCT_best_krigng_model = "SPHERICAL"
MMCT_target_field_QT = "QT_MMCT_SD_TOC_Mt_km2_"
MMCT_target_field = "MMCT_SD_TOC_Mt_km2_"
mae, rmse, r2 = test_evaluate_kriging(input_feature_class_train_OK, input_feature_class_test, MMCT_target_field_QT, MMCT_target_field, MMCT_best_krigng_model)

EXPONENTIAL model created based on training data.
EXPONENTIAL Evaluation on Test Data:MCO_SD_TOC_Mt_km2_
MAE: 2.9576, RMSE: 4.8430, R2: 0.5967
SPHERICAL model created based on training data.
SPHERICAL Evaluation on Test Data:MMCT_SD_TOC_Mt_km2_
MAE: 1.0238, RMSE: 1.6466, R2: 0.4988


In [16]:
# Evaluation function for the best IDW model.
def test_evaluate_idw(input_feature_class_train, input_feature_class_test, target_field, decay_power):
    try:

        idw_result = arcpy.sa.Idw(input_feature_class_train, target_field, power=decay_power)
        print(f"IDW model created with decay power {decay_power} based on training data.")


        predictions = []
        actual_values = []

        with arcpy.da.SearchCursor(input_feature_class_test, ["OBJECTID", "SHAPE@", target_field]) as test_cursor:
            for test_point in test_cursor:
                test_location = test_point[1].centroid
                actual_value = test_point[2]
                actual_values.append(actual_value)


                predicted_value = float(
                    arcpy.GetCellValue_management(
                        idw_result, f"{test_location.X} {test_location.Y}", "1"
                    ).getOutput(0)
                )
                predictions.append(predicted_value)

        predictions = np.array(predictions)
        actual_values = np.array(actual_values)

        mae = np.mean(np.abs(predictions - actual_values))
        rmse = np.sqrt(np.mean((predictions - actual_values) ** 2))
        sst = np.sum((actual_values - np.mean(actual_values)) ** 2)
        sse = np.sum((predictions - actual_values) ** 2)
        r2 = 1 - (sse / sst)

        print(f"IDW Evaluation on Test Data:{target_field}")
        print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

        return mae, rmse, r2

    except Exception as e:
        print(f"Error during IDW evaluation: {e}")
        return None, None, None

In [17]:
# Evaluate the final MCO result of IDW, based on the previously obtained best IDW model for MCO using 10-fold CV, with the decay power of 2.
MCO_best_decay_power = 2   
mae, rmse, r2 = test_evaluate_idw(input_feature_class_train, input_feature_class_test, MCO_target_field, MCO_best_decay_power)

# Evaluate the final MMCT result of IDW, based on the previously obtained best IDW model for MMCT using 10-fold CV, with the decay power of 1.
MMCT_best_decay_power = 1   
mae, rmse, r2 = test_evaluate_idw(input_feature_class_train, input_feature_class_test, MMCT_target_field, MMCT_best_decay_power)

IDW model created with decay power 2 based on training data.
IDW Evaluation on Test Data:MCO_SD_TOC_Mt_km2_
MAE: 3.1013, RMSE: 4.9794, R2: 0.5737
IDW model created with decay power 1 based on training data.
IDW Evaluation on Test Data:MMCT_SD_TOC_Mt_km2_
MAE: 1.0747, RMSE: 1.6594, R2: 0.4910


In [4]:
# Evaluation function for the best RF model.
# First, create a KDTree for the entire dataset to facilitate the calculation of neighboring points' values and distances.
whole_data_class = "whole_data_points"
whole_fields = ["Longitude___", "Latitude___","MCO_SD_TOC_Mt_km2_","MMCT_SD_TOC_Mt_km2_"]
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)
def test_evaluate_random_forest(input_feature_class_train, input_feature_class_test, target_field, params, whole_df,whole_tree, k):
    try:

        fields = ["Longitude___", "Latitude___", "Distance_shoreline_m", target_field]

        train_data = [row for row in arcpy.da.SearchCursor(input_feature_class_train, fields)]
        test_data = [row for row in arcpy.da.SearchCursor(input_feature_class_test, fields)]

        df_train = pd.DataFrame(train_data, columns=fields)
        df_test = pd.DataFrame(test_data, columns=fields)

        X_train = df_train.drop(columns=[target_field])
        y_train = df_train[target_field]
        X_test = df_test.drop(columns=[target_field])
        y_test = df_test[target_field]


        train_coords = X_train[["Longitude___", "Latitude___"]].values

        for idx in range(len(X_train)):
            distances, indices = whole_tree.query(train_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  # Exclude the point inself
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_train.loc[idx, f"distance_{j}"] = dist
                X_train.loc[idx, f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        test_coords = X_test[["Longitude___", "Latitude___"]].values
        for idx, test_point in enumerate(test_coords):
            test_distances, test_indices = whole_tree.query(test_point.reshape(1, -1), k=k+1)
            test_distances, test_indices = test_distances[0][1:], test_indices[0][1:]
            for j, (dist, neighbor_idx) in enumerate(zip(test_distances, test_indices), start=1):
                X_test.loc[idx, f"distance_{j}"] = dist
                X_test.loc[idx, f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        model = RandomForestRegressor(random_state=42,**params)
        model.fit(X_train, y_train)


        predictions = model.predict(X_test)


        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        sst = np.sum((y_test - np.mean(y_test)) ** 2)
        sse = np.sum((predictions - y_test) ** 2)
        r2 = 1 - (sse / sst)

        print(f"Random Forest Evaluation on Test Data:{target_field}")
        print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

        return mae, rmse, r2
    except Exception as e:
        print(f"Error during RF evaluation: {e}")
        return None, None, None

In [7]:

input_feature_class_train = "train_data_point_SpatialJoin"  
input_feature_class_test = "test_data_point_SpatialJoin"   
MCO_target_field = "MCO_SD_TOC_Mt_km2_" 
MMCT_target_field = "MMCT_SD_TOC_Mt_km2_"
# Evaluate the final RF results for MCO, based on the best RF model obtained through 10-fold CV on the training data.
# The optimal RF model for MCO is: n_estimators=100, max_features=20
MCO_best_params = {"n_estimators": 100, "max_features": 20} 
k = 25 
mae, rmse, r2 = test_evaluate_random_forest(input_feature_class_train, input_feature_class_test, MCO_target_field, MCO_best_params, whole_df, whole_tree, k)

# Evaluate the final RF results for MMCT, based on the best RF model obtained through 10-fold CV on the training data.
# The optimal RF model for MMCT is: n_estimators=400, max_features=10
MMCT_best_params = {"n_estimators": 400, "max_features": 10} 
k = 25 
mae, rmse, r2 = test_evaluate_random_forest(input_feature_class_train, input_feature_class_test, MMCT_target_field, MMCT_best_params, whole_df, whole_tree, k)

Random Forest Evaluation on Test Data:MCO_SD_TOC_Mt_km2_
MAE: 3.0511, RMSE: 4.8498, R2: 0.5956
Random Forest Evaluation on Test Data:MMCT_SD_TOC_Mt_km2_
MAE: 1.1114, RMSE: 1.6037, R2: 0.5246


# The following code is for plotting the spatial interpolation map of OK.
Since it involves QT transformation and inverse transformation, the plot is generated through the code.


In [5]:
from sklearn.preprocessing import QuantileTransformer

input_feature_class = "train_data_points"
target_field_MCO = "MCO_SD_TOC_Mt_km2_"
target_field_MMCT = "MMCT_SD_TOC_Mt_km2_"


target_fields = [target_field_MCO, target_field_MMCT]


transformed_points = {}
quantile_transformers = {}


for target_field in target_fields:
    print(f"Processing field: {target_field}")


    points = [(row[0], row[1], row[2].centroid) for row in arcpy.da.SearchCursor(input_feature_class, ["OBJECTID", target_field, "SHAPE@"])]
    actual_values = np.array([point[1] for point in points]).reshape(-1, 1)


    qt = QuantileTransformer(output_distribution="normal", random_state=42)
    transformed_values = qt.fit_transform(actual_values).flatten()


    qt_field_name = f"QT_{target_field}"
    if qt_field_name not in [field.name for field in arcpy.ListFields(input_feature_class)]:
        arcpy.AddField_management(input_feature_class, qt_field_name, "DOUBLE")


    with arcpy.da.UpdateCursor(input_feature_class, ["OBJECTID", qt_field_name]) as cursor:
        for row in cursor:
            obj_id = row[0]
            transformed_value = next((val for oid, val in zip([p[0] for p in points], transformed_values) if oid == obj_id), None)
            if transformed_value is not None:
                row[1] = transformed_value
                cursor.updateRow(row)


    quantile_transformers[target_field] = qt
    transformed_points[target_field] = [(p[0], p[1], p[2], val) for p, val in zip(points, transformed_values)]

print("Quantile Transformation completed.")



Processing field: MCO_SD_TOC_Mt_km2_


G:\Arcgis_pro\bin\Python\envs\arcgispro-py3-clone\Lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (329). n_quantiles is set to n_samples.
  warnings.warn(


Processing field: MMCT_SD_TOC_Mt_km2_


G:\Arcgis_pro\bin\Python\envs\arcgispro-py3-clone\Lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (329). n_quantiles is set to n_samples.
  warnings.warn(


Quantile Transformation completed.


In [21]:
import arcpy
import numpy as np

def modify_raster_in_place(input_raster, output_raster, quantile_transformer):
    """
    First, to simplify our code, we used ArcGIS's Kriging tool to generate the OK interpolation results for QT-SDTOC in advance.
    Then, we performed the QT-inverse transformation based on these results to get the final result.
    Copy the original raster file and directly modify its values.
    :param input_raster: Path to the original raster
    :param output_raster: Path to the output raster (the copied file)
    :param quantile_transformer: QuantileTransformer object for inverse transformation
    """
    try:

        arcpy.management.CopyRaster(input_raster, output_raster, pixel_type="32_BIT_FLOAT")
        print(f"Raster copied to: {output_raster}")

    
        raster = arcpy.Raster(output_raster)
        raster_array = arcpy.RasterToNumPyArray(raster, nodata_to_value=np.nan)
        
        reshaped_array = raster_array.reshape(-1, 1)  
        transformed_array = quantile_transformer.inverse_transform(reshaped_array)
        
        
        back_transformed_array = transformed_array.reshape(raster_array.shape)

        lower_left = arcpy.Point(raster.extent.XMin, raster.extent.YMin)

        arcpy.NumPyArrayToRaster(
            back_transformed_array,
            lower_left,
            raster.meanCellWidth,
            raster.meanCellHeight,
            value_to_nodata=np.nan
        ).save(output_raster)

        print(f"Raster values updated and saved in: {output_raster}")

    except Exception as e:
        print(f"Error in modify_raster_in_place: {e}")

In [22]:
input_feature_class = "train_data_points"
target_field_MCO = "MCO_SD_TOC_Mt_km2_"
target_field_MMCT = "MMCT_SD_TOC_Mt_km2_"

input_raster_MCO = "MCO_QT_Kriging"  
output_raster_MCO = "MCO_BackTransformed_Kriging"  
quantile_transformer_MCO = quantile_transformers[target_field_MCO]

input_raster_MMCT = "MMCT_QT_Kriging"  
output_raster_MMCT = "MMCT_BackTransformed_Kriging"  
quantile_transformer_MMCT = quantile_transformers[target_field_MMCT]

# Perform inverse transformation
modify_raster_in_place(input_raster_MCO, output_raster_MCO, quantile_transformer_MCO)
modify_raster_in_place(input_raster_MMCT, output_raster_MMCT, quantile_transformer_MMCT)

Raster values updated and saved in: MCO_BackTransformed_Kriging
Raster values updated and saved in: MMCT_BackTransformed_Kriging



# The following code is for plotting the RF spatial interpolation map, primarily by obtaining the corresponding predicted values at the fishnet points.

In [ ]:

import arcpy
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KDTree

# Set training data and fishing point data
train_feature_class = "train_data_point_SpatialJoin"
fishnet_feature_class = "fishnet_points"
whole_data_class = "whole_data_points"


target_fields = ["MCO_SD_TOC_Mt_km2_", "MMCT_SD_TOC_Mt_km2_"]
common_fields = ["Longitude___", "Latitude___", "Distance_shoreline_m"]


whole_fields = common_fields + target_fields
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)


def predict_on_fishnet(train_feature_class, fishnet_feature_class, target_field, params, whole_df, whole_tree, k=25):
    """
    The spatial interpolation prediction is carried out on the fishnet points.
    """
    try:

        train_data = [row for row in arcpy.da.SearchCursor(train_feature_class, common_fields + [target_field])]
        df_train = pd.DataFrame(train_data, columns=common_fields + [target_field])


        X_train = df_train[common_fields]
        y_train = df_train[target_field]


        train_coords = X_train[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_train)):
            distances, indices = whole_tree.query(train_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_train.loc[idx, f"distance_{j}"] = dist
                X_train.loc[idx, f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_train, y_train)


        fishnet_data = [row for row in arcpy.da.SearchCursor(fishnet_feature_class, common_fields)]
        df_fishnet = pd.DataFrame(fishnet_data, columns=common_fields)


        fishnet_coords = df_fishnet[["Longitude___", "Latitude___"]].values
        for idx, fishnet_point in enumerate(fishnet_coords):
            distances, indices = whole_tree.query(fishnet_point.reshape(1, -1), k=k)
            for j, (dist, neighbor_idx) in enumerate(zip(distances[0], indices[0]), start=1):
                df_fishnet.loc[idx, f"distance_{j}"] = dist
                df_fishnet.loc[idx, f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]

        all_tree_predictions = np.array([tree.predict(df_fishnet) for tree in model.estimators_])
        predictions = all_tree_predictions.mean(axis=0)
        std_devs = all_tree_predictions.std(axis=0)


        prediction_field = f"Prediction_{target_field}"
        uncertainty_field = f"Uncertainty_{target_field}"

        for field in [prediction_field, uncertainty_field]:
            if field not in [f.name for f in arcpy.ListFields(fishnet_feature_class)]:
                arcpy.AddField_management(fishnet_feature_class, field, "DOUBLE")

        with arcpy.da.UpdateCursor(fishnet_feature_class, common_fields + [prediction_field, uncertainty_field]) as cursor:
            for i, row in enumerate(cursor):
                row[-2] = predictions[i]
                row[-1] = std_devs[i]
                cursor.updateRow(row)

        print(f"Predictions and uncertainties for {target_field} saved in {fishnet_feature_class}")

    except Exception as e:
        print(f"Error in predict_with_uncertainty for {target_field}: {e}")

# 设置随机森林参数
params_MCO = {"n_estimators": 100, "max_features": 20}
params_MMCT = {"n_estimators": 400, "max_features": 10}

# RF predictions for MCO and MMCT
predict_on_fishnet(train_feature_class, fishnet_feature_class, target_fields[0], params_MCO, whole_df, whole_tree)
predict_on_fishnet(train_feature_class, fishnet_feature_class, target_fields[1], params_MMCT, whole_df, whole_tree)

In [ ]:
"""
It is to re-optimization hyperparameters specific to SDTOC, so that spatial interpolation results can be obtained based on all the whole data.(Section 3.4 in the manuscript)
"""

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KDTree
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import arcpy


whole_data_class = "whole_data_points"
whole_fields = ["Longitude___", "Latitude___", "MCO_SD_TOC_Mt_km2_", "MMCT_SD_TOC_Mt_km2_"]
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)

def kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k, n_splits=10):


    fields = ["Longitude___", "Latitude___", "Distance_shoreline_m", target_field]
    data = [row for row in arcpy.da.SearchCursor(input_feature_class, fields)]
    df = pd.DataFrame(data, columns=fields)


    X = df.drop(columns=[target_field])
    y = df[target_field]


    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)


    predictions = []
    actual_values = []


    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Processing fold {fold + 1}/{n_splits}...")
        

        X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
        y_train, y_val = y.iloc[train_idx].copy(), y.iloc[val_idx].copy()


        train_coords = X_train[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_train)):
            distances, indices = whole_tree.query(train_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_train.loc[X_train.index[idx], f"distance_{j}"] = dist
                X_train.loc[X_train.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        val_coords = X_val[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_val)):
            distances, indices = whole_tree.query(val_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_val.loc[X_val.index[idx], f"distance_{j}"] = dist
                X_val.loc[X_val.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_train, y_train)


        y_pred = model.predict(X_val)
        predictions.extend(y_pred)
        actual_values.extend(y_val)


    mae = mean_absolute_error(actual_values, predictions)
    rmse = np.sqrt(mean_squared_error(actual_values, predictions))
    r2 = r2_score(actual_values, predictions)

    return mae, rmse, r2


In [5]:
# target is MCO
input_feature_class = "whole_data_points"
target_field = "MCO_SD_TOC_Mt_km2_"


results = []


param_grid = {
    "n_estimators": list(range(50, 501, 50)),  
    "max_features": list(range(5, 54, 5))  
}


for params in ParameterGrid(param_grid):
    print(f"Training with n_estimators: {params['n_estimators']}, max_features: {params['max_features']}")

    mae, rmse, r2 = kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k=25, n_splits=10)

    results.append({
        'n_estimators': params['n_estimators'],
        'max_features': params['max_features'],
        'avg_mae': mae,
        'avg_rmse': rmse,
        'avg_r2': r2
    })

    print(f"Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

df_results = pd.DataFrame(results)
output_file = "1212_RandomForest_10FoldCV_Results_MCO_25临近点_基于全部数据.csv"
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

Training with n_estimators: 50, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 3.0332, RMSE: 4.6122, R2: 0.5957
Training with n_estimators: 100, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.9894, RMSE: 4.5604, R2: 0.6047
Training with n_estimators: 150, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.9719, RMSE: 4.5578

Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.8782, RMSE: 4.4968, R2: 0.6156
Training with n_estimators: 500, max_features: 25
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.8815, RMSE: 4.5027, R2: 0.6146
Training with n_estimators: 50, max_features: 30
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.9177, RMSE: 4.4916, R2: 0.6165
Training with n_estimators: 100, max_features: 30
Processing fold 1/10...
Processi

Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.8823, RMSE: 4.5296, R2: 0.6100
Training with n_estimators: 400, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.8881, RMSE: 4.5383, R2: 0.6085
Training with n_estimators: 450, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 2.8905, RMSE: 4.5420, R2: 0.6079
Training with n_estimators: 500, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Process

In [6]:
# target is MMCT
input_feature_class = "whole_data_points"
target_field = "MMCT_SD_TOC_Mt_km2_"


results = []


param_grid = {
    "n_estimators": list(range(50, 501, 50)),  
    "max_features": list(range(5, 54, 5))  
}


for params in ParameterGrid(param_grid):
    print(f"Training with n_estimators: {params['n_estimators']}, max_features: {params['max_features']}")

    mae, rmse, r2 = kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k=25, n_splits=10)

    results.append({
        'n_estimators': params['n_estimators'],
        'max_features': params['max_features'],
        'avg_mae': mae,
        'avg_rmse': rmse,
        'avg_r2': r2
    })

    print(f"Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


df_results = pd.DataFrame(results)
output_file = "1211_RandomForest_10FoldCV_Results_MMCT_25临近点_基于全部数据.csv"
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

Training with n_estimators: 50, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.2238, RMSE: 1.8864, R2: 0.4576
Training with n_estimators: 100, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.2099, RMSE: 1.8840, R2: 0.4590
Training with n_estimators: 150, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.2165, RMSE: 1.9019

Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.1622, RMSE: 1.8677, R2: 0.4683
Training with n_estimators: 500, max_features: 25
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.1661, RMSE: 1.8711, R2: 0.4664
Training with n_estimators: 50, max_features: 30
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.1841, RMSE: 1.8929, R2: 0.4539
Training with n_estimators: 100, max_features: 30
Processing fold 1/10...
Processi

Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.1562, RMSE: 1.8695, R2: 0.4673
Training with n_estimators: 400, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.1588, RMSE: 1.8682, R2: 0.4680
Training with n_estimators: 450, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 1.1615, RMSE: 1.8691, R2: 0.4675
Training with n_estimators: 500, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Process

In [ ]:
"""
Assumed that RF is the best spatial interpolation method. Obtain hyperparameters for Thickness so that the result of spatial interpolation can be obtained based on the whole data (text S2 in Supplementary Material).
"""

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KDTree
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import arcpy


whole_data_class = "whole_data_points"
whole_fields = ["Longitude___", "Latitude___", "MCO_Thickness_m_", "MMCT_Thickness_m_"]
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)

def kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k, n_splits=10):


    fields = ["Longitude___", "Latitude___", "Distance_shoreline_m", target_field]
    data = [row for row in arcpy.da.SearchCursor(input_feature_class, fields)]
    df = pd.DataFrame(data, columns=fields)


    X = df.drop(columns=[target_field])
    y = df[target_field]


    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)


    predictions = []
    actual_values = []


    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Processing fold {fold + 1}/{n_splits}...")
        

        X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
        y_train, y_val = y.iloc[train_idx].copy(), y.iloc[val_idx].copy()

        train_coords = X_train[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_train)):
            distances, indices = whole_tree.query(train_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_train.loc[X_train.index[idx], f"distance_{j}"] = dist
                X_train.loc[X_train.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        val_coords = X_val[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_val)):
            distances, indices = whole_tree.query(val_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_val.loc[X_val.index[idx], f"distance_{j}"] = dist
                X_val.loc[X_val.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        predictions.extend(y_pred)
        actual_values.extend(y_val)


    mae = mean_absolute_error(actual_values, predictions)
    rmse = np.sqrt(mean_squared_error(actual_values, predictions))
    r2 = r2_score(actual_values, predictions)

    return mae, rmse, r2


In [8]:
# target is MCO, thickness
input_feature_class = "whole_data_points"
target_field = "MCO_Thickness_m_"


results = []

param_grid = {
    "n_estimators": list(range(50, 501, 50)),  
    "max_features": list(range(5, 54, 5))  
}


for params in ParameterGrid(param_grid):
    print(f"Training with n_estimators: {params['n_estimators']}, max_features: {params['max_features']}")

    mae, rmse, r2 = kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k=25, n_splits=10)

    results.append({
        'n_estimators': params['n_estimators'],
        'max_features': params['max_features'],
        'avg_mae': mae,
        'avg_rmse': rmse,
        'avg_r2': r2
    })

    print(f"Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


df_results = pd.DataFrame(results)
output_file = "1212_RandomForest_10FoldCV_Thickness_MCO_25临近点_基于全部数据.csv"
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

Training with n_estimators: 50, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 144.5975, RMSE: 235.5697, R2: 0.4423
Training with n_estimators: 100, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 142.0351, RMSE: 231.0537, R2: 0.4635
Training with n_estimators: 150, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 140.6182, RM

Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 136.3119, RMSE: 228.3668, R2: 0.4759
Training with n_estimators: 450, max_features: 25
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 136.2588, RMSE: 228.4374, R2: 0.4756
Training with n_estimators: 500, max_features: 25
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 136.0846, RMSE: 228.4996, R2: 0.4753
Training with n_estimators: 50, max_features: 30
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10

Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 136.7939, RMSE: 230.4348, R2: 0.4664
Training with n_estimators: 350, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 136.6865, RMSE: 230.2444, R2: 0.4673
Training with n_estimators: 400, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 136.6678, RMSE: 230.4180, R2: 0.4665
Training with n_estimators: 450, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/1

In [9]:
# target is MMCT, thickness
input_feature_class = "whole_data_points"
target_field = "MMCT_Thickness_m_"


results = []


param_grid = {
    "n_estimators": list(range(50, 501, 50)),  
    "max_features": list(range(5, 54, 5))  
}


for params in ParameterGrid(param_grid):
    print(f"Training with n_estimators: {params['n_estimators']}, max_features: {params['max_features']}")

    mae, rmse, r2 = kfold_cv_random_forest(input_feature_class, target_field, params, whole_df, whole_tree, k=25, n_splits=10)

    results.append({
        'n_estimators': params['n_estimators'],
        'max_features': params['max_features'],
        'avg_mae': mae,
        'avg_rmse': rmse,
        'avg_r2': r2
    })

    print(f"Results - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


df_results = pd.DataFrame(results)
output_file = "1212_RandomForest_10FoldCV_Thickness_MMCT_25临近点_基于全部数据.csv"
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

Training with n_estimators: 50, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 96.5167, RMSE: 141.4636, R2: 0.3351
Training with n_estimators: 100, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 96.5720, RMSE: 141.6286, R2: 0.3336
Training with n_estimators: 150, max_features: 5
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 96.6605, RMSE:

Processing fold 10/10...
Results - MAE: 95.7852, RMSE: 139.6508, R2: 0.3520
Training with n_estimators: 450, max_features: 25
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 95.8209, RMSE: 139.7904, R2: 0.3507
Training with n_estimators: 500, max_features: 25
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 95.8566, RMSE: 139.9121, R2: 0.3496
Training with n_estimators: 50, max_features: 30
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...

Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 94.6978, RMSE: 139.6037, R2: 0.3525
Training with n_estimators: 350, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 94.7278, RMSE: 139.7356, R2: 0.3513
Training with n_estimators: 400, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10...
Processing fold 7/10...
Processing fold 8/10...
Processing fold 9/10...
Processing fold 10/10...
Results - MAE: 94.9380, RMSE: 140.1697, R2: 0.3472
Training with n_estimators: 450, max_features: 50
Processing fold 1/10...
Processing fold 2/10...
Processing fold 3/10...
Processing fold 4/10...
Processing fold 5/10...
Processing fold 6/10..

In [ ]:
"""
Importance assessment. Just save the top ten important parameters.
"""

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KDTree
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import arcpy


whole_data_class = "whole_data_points"
whole_fields = ["Longitude___", "Latitude___", "MCO_SD_TOC_Mt_km2_", "MMCT_SD_TOC_Mt_km2_"]
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)

def calculate_feature_importance(input_feature_class, target_field, params, whole_df, whole_tree, k):


    fields = ["Longitude___", "Latitude___", "Distance_shoreline_m", target_field]
    data = [row for row in arcpy.da.SearchCursor(input_feature_class, fields)]
    df = pd.DataFrame(data, columns=fields)


    X = df.drop(columns=[target_field])
    y = df[target_field]

    coords = X[["Longitude___", "Latitude___"]].values
    for idx in range(len(X)):
        distances, indices = whole_tree.query(coords[idx].reshape(1, -1), k=k + 1)
        distances, indices = distances[0][1:], indices[0][1:]  
        for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
            X.loc[X.index[idx], f"distance_{j}"] = dist
            X.loc[X.index[idx], f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


    model = RandomForestRegressor(random_state=42, **params)
    model.fit(X, y)


    feature_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    return feature_importances

In [45]:

input_feature_class = "whole_data_points"
MCO_target_field = "MCO_SD_TOC_Mt_km2_"
MMCT_target_field = "MMCT_SD_TOC_Mt_km2_"

MCO_best_params = {
    "n_estimators": 450,  
    "max_features": 10  
}

MMCT_best_params = {
    "n_estimators": 400,  
    "max_features": 25  
}


mco_importances = calculate_feature_importance(input_feature_class, MCO_target_field, MCO_best_params, whole_df, whole_tree, k=25)
mmct_importances = calculate_feature_importance(input_feature_class, MMCT_target_field, MMCT_best_params, whole_df, whole_tree, k=25)

with pd.ExcelWriter("20241219_Feature_Importances.xlsx") as writer:
    mco_importances.to_excel(writer, sheet_name="MCO_Importance", index=False)
    mmct_importances.to_excel(writer, sheet_name="MMCT_Importance", index=False)

    
print("MCO Feature Importances:")
print(mco_importances)
print("\nMMCT Feature Importances:")
print(mmct_importances)

MCO Feature Importances:
                  Feature  Importance
4    MCO_SD_TOC_Mt_km2__1    0.140134
6    MCO_SD_TOC_Mt_km2__2    0.119939
14   MCO_SD_TOC_Mt_km2__6    0.073712
8    MCO_SD_TOC_Mt_km2__3    0.061291
10   MCO_SD_TOC_Mt_km2__4    0.058444
12   MCO_SD_TOC_Mt_km2__5    0.042182
16   MCO_SD_TOC_Mt_km2__7    0.038496
24  MCO_SD_TOC_Mt_km2__11    0.029708
20   MCO_SD_TOC_Mt_km2__9    0.027565
22  MCO_SD_TOC_Mt_km2__10    0.024342
2    Distance_shoreline_m    0.021268
40  MCO_SD_TOC_Mt_km2__19    0.021228
18   MCO_SD_TOC_Mt_km2__8    0.020670
38  MCO_SD_TOC_Mt_km2__18    0.019070
32  MCO_SD_TOC_Mt_km2__15    0.018810
28  MCO_SD_TOC_Mt_km2__13    0.015626
0            Longitude___    0.015215
36  MCO_SD_TOC_Mt_km2__17    0.014282
42  MCO_SD_TOC_Mt_km2__20    0.013216
52  MCO_SD_TOC_Mt_km2__25    0.013012
1             Latitude___    0.011872
50  MCO_SD_TOC_Mt_km2__24    0.011388
26  MCO_SD_TOC_Mt_km2__12    0.011379
44  MCO_SD_TOC_Mt_km2__21    0.010119
46  MCO_SD_TOC_Mt_km2__22

In [ ]:
"""
The following is the result of the random forest spatial interpolation based on all the whole data, including SDTOC and Thickness, drawn during MCO and MMCT.
"""

In [ ]:

import arcpy
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KDTree


train_feature_class = "whole_data_points"
fishnet_feature_class = "fishnet_points"
whole_data_class = "whole_data_points"

target_fields = ["MCO_SD_TOC_Mt_km2_", "MMCT_SD_TOC_Mt_km2_", "MCO_Thickness_m_","MMCT_Thickness_m_"]
common_fields = ["Longitude___", "Latitude___", "Distance_shoreline_m"]


whole_fields = common_fields + target_fields
whole_data = [row for row in arcpy.da.SearchCursor(whole_data_class, whole_fields)]
whole_df = pd.DataFrame(whole_data, columns=whole_fields)
whole_coords = whole_df[["Longitude___", "Latitude___"]].values
whole_tree = KDTree(whole_coords)


def predict_on_fishnet(train_feature_class, fishnet_feature_class, target_field, params, whole_df, whole_tree, k=25):
    """
    在渔网点上进行空间插值预测。
    """
    try:

        train_data = [row for row in arcpy.da.SearchCursor(train_feature_class, common_fields + [target_field])]
        df_train = pd.DataFrame(train_data, columns=common_fields + [target_field])

        X_train = df_train[common_fields]
        y_train = df_train[target_field]


        train_coords = X_train[["Longitude___", "Latitude___"]].values
        for idx in range(len(X_train)):
            distances, indices = whole_tree.query(train_coords[idx].reshape(1, -1), k=k + 1)
            distances, indices = distances[0][1:], indices[0][1:]  
            for j, (dist, neighbor_idx) in enumerate(zip(distances, indices), start=1):
                X_train.loc[idx, f"distance_{j}"] = dist
                X_train.loc[idx, f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_train, y_train)

        fishnet_data = [row for row in arcpy.da.SearchCursor(fishnet_feature_class, common_fields)]
        df_fishnet = pd.DataFrame(fishnet_data, columns=common_fields)


        fishnet_coords = df_fishnet[["Longitude___", "Latitude___"]].values
        for idx, fishnet_point in enumerate(fishnet_coords):
            distances, indices = whole_tree.query(fishnet_point.reshape(1, -1), k=k)
            for j, (dist, neighbor_idx) in enumerate(zip(distances[0], indices[0]), start=1):
                df_fishnet.loc[idx, f"distance_{j}"] = dist
                df_fishnet.loc[idx, f"{target_field}_{j}"] = whole_df.iloc[neighbor_idx][target_field]


        all_tree_predictions = np.array([tree.predict(df_fishnet) for tree in model.estimators_])
        predictions = all_tree_predictions.mean(axis=0)
        std_devs = all_tree_predictions.std(axis=0)


        prediction_field = f"Prediction_{target_field}"
        uncertainty_field = f"Uncertainty_{target_field}"

        for field in [prediction_field, uncertainty_field]:
            if field not in [f.name for f in arcpy.ListFields(fishnet_feature_class)]:
                arcpy.AddField_management(fishnet_feature_class, field, "DOUBLE")

        with arcpy.da.UpdateCursor(fishnet_feature_class, common_fields + [prediction_field, uncertainty_field]) as cursor:
            for i, row in enumerate(cursor):
                row[-2] = predictions[i]
                row[-1] = std_devs[i]
                cursor.updateRow(row)

        print(f"Predictions and uncertainties for {target_field} saved in {fishnet_feature_class}")

    except Exception as e:
        print(f"Error in predict_with_uncertainty for {target_field}: {e}")


params_MCO_SDTOC = {"n_estimators": 450, "max_features": 10}
params_MMCT_SDTOC = {"n_estimators": 400, "max_features": 25}
params_MCO_Thickness = {"n_estimators": 100, "max_features": 15}
params_MMCT_Thickness = {"n_estimators": 50, "max_features": 35}


predict_on_fishnet(train_feature_class, fishnet_feature_class, target_fields[0], params_MCO_SDTOC, whole_df, whole_tree)
predict_on_fishnet(train_feature_class, fishnet_feature_class, target_fields[1], params_MMCT_SDTOC, whole_df, whole_tree)
predict_on_fishnet(train_feature_class, fishnet_feature_class, target_fields[2], params_MCO_Thickness, whole_df, whole_tree)
predict_on_fishnet(train_feature_class, fishnet_feature_class, target_fields[3], params_MMCT_Thickness, whole_df, whole_tree)